<!-- :Author: Arthur Goldberg <Arthur.Goldberg@mssm.edu> -->
<!-- :Date: 2020-08-02 -->
<!-- :Copyright: 2020, Karr Lab -->
<!-- :License: MIT -->
# DE-Sim: Ordering simultaneous events

DE-Sim makes it easy to build and simulate discrete-event models.
This notebook discusses DE-Sim's methods for controlling the execution order of simultaneous messages.

## Installation
Use `pip` to install `de_sim`.

## Scheduling events with equal simulation times

A discrete-event simulation may execute multiple events simultaneously, that is, at a particular simulation time.
To ensure that simulation runs are reproducible and deterministic, a simulator must provide mechanisms that deterministically control the execution order of simultaneous events.

Two types of situations arise, *local* and *global*.
A local situation arises when a simulation object receives multiple event messages simultaneously, while a global situation arises when multiple simulation objects execute events simultaneously.

Separate *local* and *global* mechanisms ensure that these situations are simulated deterministically.
The local mechanism ensures that simultaneous events are handled deterministically at a single simulation object, while the global mechanism ensures that simultaneous events are handled deterministically across all objects in a simulation.

### Local mechanism: simultaneous event messages at a simulation object
The local mechanism, called *event superposition* after the [physics concept of superposition](https://en.wikipedia.org/wiki/Superposition_principle), involves two components:

1. When a simulation object receives multiple event messages at the same time, the simulator passes all of the event messages to the object's event handler in a list.
(However, if simultaneous event messages have different handlers then the simulator raises a `SimulatorError` exception.)
2. The simulator sorts the events in the list so that any given list of events will always be arranged in the same order.

Event messages are sorted by the pair (event message priority, event message content).
Sorting costs O(n log n), but since simultaneous events are usually rare, sorting event lists is unlikely to slow down simulations.

In [2]:
""" This example illustrates the local mechanism that handles simultaneous
    event messages received by a simulation object
"""
import random

import de_sim
from de_sim.event import Event

class Double(de_sim.EventMessage):
    'Double value'

class Increment(de_sim.EventMessage):
    'Increment value'

class IncrementThenDoubleSimObject(de_sim.SimulationObject):
    """ Execute Increment before Double, demonstrating superposition """

    def __init__(self, name):
        super().__init__(name)
        self.value = 0

    def init_before_run(self):
        self.send_events()

    def handle_superposed_events(self, event_list):
        """ Process superposed events in an event list

        Each Increment message increments value, and each Double message doubles value.
        Assumes that `event_list` contains an Increment event followed by a Double event.

        Args:
            event_list (:obj:`event_list` of :obj:`de_sim.Event`): list of events
        """
        for event in event_list:
            if isinstance(event.message, Increment):
                self.value += 1
            elif isinstance(event.message, Double):
                self.value *= 2
        self.send_events()

    # The order of the message types in event_handlers, (Increment, Double), determines
    # the sort order of messages in `event_list` received by `handle_superposed_events`
    event_handlers = [(Increment, 'handle_superposed_events'),
                      (Double, 'handle_superposed_events')]

    def send_events(self):
        # To show that the simulator delivers event messages to `handle_superposed_events`
        # sorted into the order (Increment, Double), send them in a random order.
        if random.randrange(2):
            self.send_event(1, self, Double())
            self.send_event(1, self, Increment())
        else:
            self.send_event(1, self, Increment())
            self.send_event(1, self, Double())

    # Register the message types sent
    messages_sent = (Increment, Double)


class TestSuperposition(object):

    def increment_then_double_from_0(self, iterations):
        v = 0
        for _ in range(iterations):
            v += 1
            v *= 2
        return v

    def test_superposition(self, max_time):
        simulator = de_sim.Simulator()
        simulator.add_object(IncrementThenDoubleSimObject('name'))
        simulator.initialize()
        simulator.simulate(max_time)
        for sim_obj in simulator.get_objects():
            assert sim_obj.value, self.increment_then_double_from_0(max_time)
        print(f'Simulation to {max_time} executed all messages in the order (Increment, Double).')

TestSuperposition().test_superposition(20)

Simulation to 20 executed all messages in the order (Increment, Double).


This example shows how event superposition handles simultaneous events.
An `IncrementThenDoubleSimObject` simulation object stores an integer value.
It receives two events every time unit, one carrying an `Increment` message and another containing a `Double` message.
Executing an `Increment` event increments the value, while executing a `Double` message event doubles the value.
The design for `IncrementThenDoubleSimObject` requires that it increments before doubling.

Several features of DE-Sim and `IncrementThenDoubleSimObject` ensure this behavior:

1. The mapping between event message types and event handlers, stored in the list `event_handlers`, contains `Increment` before `Double`. This gives events containing an `Increment` message a higher priority than events containing `Double`.
2. Under the covers, when DE-Sim passes superposed events to a subclass of [`SimulationObject`](https://docs.karrlab.org/de_sim/master/source/de_sim.html#de_sim.simulation_object.SimulationObject), it sorts the messages by their (event message priority, event message content), which sorts events with higher priority message types earlier.
3. The message handler `handle_superposed_events` receives a list of events and executes them in order.

To challenge and test this superposition mechanism, the `send_events()` method in `IncrementThenDoubleSimObject` randomizes the order in which it sends `Increment` and `Double` events.
Finally, `TestSuperposition().test_superposition()` runs a simulation of `IncrementThenDoubleSimObject` and asserts that the value it computes equals the correct value for a sequence of increment and double operations.

### Global mechanism: simultaneous event messages at multiple simulation objects
A *global* mechanism is needed to ensure that simultaneous events which occur at distinct objects in a simulation are executed in a deterministic order.
Otherwise, the discrete-event simulator might execute simultaneous events at distinct simulation objects in a different order in different simulation runs that use the same input.
When using a simulator that allows 0-delay event messages or global state shared between simulation objects -- both of which DE-Sim supports -- this can alter the simulation's predictions and thereby imperil debugging efforts, statistical analyses of predictions and other essential uses of simulation results.

The global mechanism employed by DE-Sim conceives of the simulation time as a pair -- the event time, and a *sub-time* which breaks event time ties.
Sub-time values within a particular simulation time must be distinct.
Given that constraint, many approaches for selecting the sub-time would achieve the objective.
DE-Sim creates a distinct sub-time from the state of the simulation object receiving an event.

The sub-time is a pair composed of a priority assigned to the simulation class and a unique identifier for each class instance.
Each simulation class defines a `class_priority` attribute that determines the relative execution order of simultaneous events by different simulation classes.
Among multiple instances of a simulation class, the attribute `event_time_tiebreaker`, which defaults to a simulation instance's unique name, breaks ties.
All classes have the same default priority of `LOW`. If class priorities are not set and  `event_time_tiebreaker`s are not set for individual simulation objects, then an object's global priority is given by its name.

In [6]:
from de_sim.simulation_object import SimObjClassPriority


class ExampleMsg(de_sim.EventMessage):
    'Example message'


class NoPrioritySimObj(de_sim.SimulationObject):

    def init_before_run(self):
        self.send_event(0., self, ExampleMsg())

    # register the message types sent
    messages_sent = (ExampleMsg, )


class LowPrioritySimObj(NoPrioritySimObj):

    def handler(self, event):
        print(f"{self.time}: LowPrioritySimObj {self.name} running")
        self.send_event(1., self, ExampleMsg())

    event_handlers = [(ExampleMsg, 'handler')]

    # have `LowPrioritySimObj`s execute at low priority
    class_priority = SimObjClassPriority.LOW


class MediumPrioritySimObj(NoPrioritySimObj):

    def handler(self, event):
        print(f"{self.time}: MediumPrioritySimObj {self.name} running")
        self.send_event(1., self, ExampleMsg())

    event_handlers = [(ExampleMsg, 'handler')]

    # have `MediumPrioritySimObj`s execute at medium priority
    class_priority = SimObjClassPriority.MEDIUM

simulator = de_sim.Simulator()
simulator.add_object(LowPrioritySimObj('A'))
simulator.add_object(MediumPrioritySimObj('B'))
simulator.initialize()
print(simulator.simulate(2).num_events, 'events executed')

0.0: MediumPrioritySimObj B running
0.0: LowPrioritySimObj A running
1.0: MediumPrioritySimObj B running
1.0: LowPrioritySimObj A running
2.0: MediumPrioritySimObj B running
2.0: LowPrioritySimObj A running
6 events executed


This example illustrates the scheduling of simultaneous event messages.
`SimObjClassPriority` is an `IntEnum` that provides simulation object class priorities, including `LOW`, `MEDIUM`, and `HIGH`.
We create two classes, `LowPrioritySimObj` and `MediumPrioritySimObj`, with `LOW` and `MEDIUM` priorities, respectively, and execute them simultaneously at simulation times 0, 1, 2, ...
At each time, the `MediumPrioritySimObj` object runs before the `LowPrioritySimObj` one.

#### Execution order of objects without an assigned `class_priority`
The next example shows the ordering of simultaneous events executed by objects that don't have assigned priorities.

In [4]:
class DefaultPrioritySimObj(NoPrioritySimObj):

    def handler(self, event):
        print(f"{self.time}: DefaultPrioritySimObj {self.name} running")
        self.send_event(1., self, ExampleMsg())

    event_handlers = [(ExampleMsg, 'handler')]


simulator = de_sim.Simulator()
for name in random.sample(range(10), k=3):
    sim_obj = DefaultPrioritySimObj(str(name))
    print(f"{sim_obj.name} priority: {sim_obj.class_event_priority.name}")
    simulator.add_object(sim_obj)
simulator.initialize()
print(simulator.simulate(2).num_events, 'events executed')

9 priority: LOW
6 priority: LOW
2 priority: LOW
0.0: DefaultPrioritySimObj 2 running
0.0: DefaultPrioritySimObj 6 running
0.0: DefaultPrioritySimObj 9 running
1.0: DefaultPrioritySimObj 2 running
1.0: DefaultPrioritySimObj 6 running
1.0: DefaultPrioritySimObj 9 running
2.0: DefaultPrioritySimObj 2 running
2.0: DefaultPrioritySimObj 6 running
2.0: DefaultPrioritySimObj 9 running
9 events executed


In this example, the [`SimulationObject`s](https://docs.karrlab.org/de_sim/master/source/de_sim.html#de_sim.simulation_object.SimulationObject) have no priorities assigned, so their default priorities are `LOW`. (The `class_event_priority` attribute of a simulation object is a `SimObjClassPriority`)

Three objects with names randomly selected from '0', '1', ..., '9', are created.
When they execute simultaneously, events are ordered by the sort order of the objects' names.

#### Execution order of instances of simulation object classes with relative priorities
Often, a modeler wants to control the *relative* simultaneous priorities of simulation objects, but does not care about their absolute priorities.
The next example shows how to specify relative priorities.

In [5]:
class FirstNoPrioritySimObj(NoPrioritySimObj):

    def handler(self, event):
        print(f"{self.time}: FirstNoPrioritySimObj {self.name} running")
        self.send_event(1., self, ExampleMsg())

    event_handlers = [(ExampleMsg, 'handler')]


class SecondNoPrioritySimObj(NoPrioritySimObj):

    def handler(self, event):
        print(f"{self.time}: SecondNoPrioritySimObj {self.name} running")
        self.send_event(1., self, ExampleMsg())

    event_handlers = [(ExampleMsg, 'handler')]


# Assign decreasing priorities to classes in [FirstNoPrioritySimObj, SecondNoPrioritySimObj]
SimObjClassPriority.assign_decreasing_priority([FirstNoPrioritySimObj,
                                                SecondNoPrioritySimObj])

simulator = de_sim.Simulator()
simulator.add_object(SecondNoPrioritySimObj('A'))
simulator.add_object(FirstNoPrioritySimObj('B'))
for sim_obj in simulator.simulation_objects.values():
    print(f"{type(sim_obj).__name__}: {sim_obj.name}; "
          f"priority: {sim_obj.class_event_priority.name}")
simulator.initialize()
print(simulator.simulate(2).num_events, 'events executed')

SecondNoPrioritySimObj: A; priority: SECOND
FirstNoPrioritySimObj: B; priority: HIGH
0.0: FirstNoPrioritySimObj B running
0.0: SecondNoPrioritySimObj A running
1.0: FirstNoPrioritySimObj B running
1.0: SecondNoPrioritySimObj A running
2.0: FirstNoPrioritySimObj B running
2.0: SecondNoPrioritySimObj A running
6 events executed


The `assign_decreasing_priority` method of `SimObjClassPriority` takes an iterator over `SimulationObject` subclasses, and assigns them decreasing simultaneous event priorities.
The `FirstNoPrioritySimObj` instance therefore executes before the `SecondNoPrioritySimObj` instance at each discrete simulation time.